In [18]:
import pandas as pd
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col , monotonically_increasing_id
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql import types as t
import re
import io



In [19]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

In [20]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
df_spark =spark.read.format('com.github.saurfang.sas.spark').load('I94_SAS_Labels_Descriptions.SAS')

In [21]:
with open("I94_SAS_Labels_Descriptions.SAS", 'r') as file_handle:
    # read file content into list
    #lines = file_handle.readlines()
    file=file_handle.read()
    #headers=next(lines)
sas_dict={}
temp_data=[]


# print list content
#print(lines)

In [24]:
for line in file.split("\n"):
        line = re.sub(r"\s+", " ", line)
        if "/*" in line and "-" in line:
            k, v = [i.strip(" ") for i in line.split("*")[1]
                                              .split("-", 1)]
            k = k.replace(' & ', '_').lower()
            sas_dict[k] = {'description': v}
        elif '=' in line and ';' not in line:
            temp_data.append([i.strip(' ').strip("'").title()
                              for i in line.split('=')])
        elif len(temp_data) > 0:
            sas_dict[k]['data'] = temp_data
            temp_data = []

sas_dict['i94cit_i94res']['df'] = pd.DataFrame(
        sas_dict['i94cit_i94res']['data'], columns=['code', 'country'])

tempdf = pd.DataFrame(sas_dict['i94port']['data'],
                          columns=['code', 'port_of_entry'])
tempdf['code'] = tempdf['code'].str.upper()
tempdf[['city', 'state_or_country']] = tempdf['port_of_entry'
                                                  ].str.rsplit(',', 1,
                                                               expand=True)
sas_dict['i94port']['df'] = tempdf

sas_dict['i94mode']['df'] = pd.DataFrame(
        sas_dict['i94mode']['data'], columns=['code', 'transportation'])

tempdf = pd.DataFrame(sas_dict['i94addr']['data'],
                          columns=['code', 'state'])
tempdf['code'] = tempdf['code'].str.upper()
sas_dict['i94addr']['df'] = tempdf

sas_dict['i94visa']['df'] = pd.DataFrame(
    
    sas_dict['i94visa']['data'], columns=['code', 'reason_for_travel'])

for table in sas_dict.keys():
    if 'df' in sas_dict[table].keys():
        with io.open(f"{table}.csv", "w" ) as f:
           sas_dict[table]['df'].to_csv(f, index=False)
            
        
                       
            
            
            